In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [3]:

RANDOM_SEED = 3971944
np.random.seed(RANDOM_SEED)

In [4]:
def read_csvFile_to_df(file_name):
    return pd.read_csv(file_name)

In [5]:
train_df = read_csvFile_to_df('TweetsMusk copy.csv')
test_df = read_csvFile_to_df('2years.csv')
test_df.head()

,Unnamed: 0,index,Unnamed: 0.1,id,conversation_id,created_at,date,timezone,place,tweet,...,reply_to,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count
0,0,0,0.0,1343644462036086785,1343320495127633920,1609184778000.0,2020-12-28 19:46:18,0,NaN,Entertainment will be critical when cars drive...,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1.0,1343619610617077760,1343386617294295040,1609178853000.0,2020-12-28 18:07:33,0,NaN,@kimpaquette Just meeting with Larry Ellison t...,...,"[{'screen_name': 'kimpaquette', 'name': 'Kim P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2.0,1343608616960491521,1343576442722893825,1609176231000.0,2020-12-28 17:23:51,0,NaN,@richierichhhhh_ Absolutely,...,"[{'screen_name': 'richierichhhhh_', 'name': 'R...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3.0,1343608530998153222,1343320495127633920,1609176211000.0,2020-12-28 17:23:31,0,NaN,What should Tesla do with in-car gaming in an ...,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4.0,1343431408052662273,1343043963096326147,1609133982000.0,2020-12-28 05:39:42,0,NaN,@PPathole @WSJ Absolutely,...,"[{'screen_name': 'PPathole', 'name': 'Pranay P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
train_df.head()

,id,User,Tweet,created_at,hashtags,lang,Label
0,0,elonmusk,"@MedvedevRussiaE @trussliz Btw, how’s it going...",2022-10-20 21:20:40+00:00,[],en,F
1,1,elonmusk,@MedvedevRussiaE @trussliz Pretty good troll tbh,2022-10-20 21:12:17+00:00,[],en,F
2,2,elonmusk,@Alan_McGuinness *humour,2022-10-20 21:07:30+00:00,[],fr,NF
3,3,elonmusk,@ilyasut I couldn’t agree more 😜,2022-10-20 20:37:33+00:00,[],en,NF
4,4,elonmusk,@Alan_McGuinness British humor is the best,2022-10-20 20:36:51+00:00,[],en,NF


In [8]:

def preprocess(data):
    data= data.lower()
    data = re.sub(r'\d+', '', data)
    data = re.sub("'", "", data)
    data = re.sub("@[A-Za-z0-9_]+","", data)
    data = re.sub("#[A-Za-z0-9_]+","", data)
    data = re.sub(r'http\S+', '', data)
    data = re.sub('[()!?]', ' ', data)
    data = re.sub('\[.*?\]',' ', data)
    data = data.strip()
    return data

def series_to_tfidf(sr, **kwargs):
    vectorizer = TfidfVectorizer(**kwargs)
    titleFeatues = vectorizer.fit_transform(sr).toarray()
    return vectorizer, titleFeatues


def evaluate_classification(y_true, y_predicted):
    print("="*50)
    print("Classification Report")
    print(classification_report(y_true, y_predicted))
    print("="*50)
    ConfusionMatrixDisplay.from_predictions(y_true, y_predicted)
    plt.xticks(rotation=90)
    plt.show()
    

In [27]:

#extracting title in to df for train and test set
trainTitleSeries = train_df['Tweet']
testTitleSeries = test_df['tweet']
# extracting labels from train_df
labels = train_df['Label']

#converting train title df to tf-idf which returns
trainTitleFeatues = series_to_tfidf(trainTitleSeries,
                                    stop_words="english",
                                    lowercase=True,
                                    vocabulary=None,
                                    preprocessor=lambda x: preprocess(x),
                                    analyzer ='word',
                                    strip_accents="unicode",
                                    min_df=5,
                                    ngram_range=(1,4),
                                    max_features = 500
                                   )

#Using Vocabulary only from Train Set
testTitleFeatues = series_to_tfidf(testTitleSeries,
                                    stop_words="english",
                                    lowercase=True,
                                    preprocessor=lambda x: preprocess(x),
                                    analyzer ='word',
                                    strip_accents="unicode",
                                    min_df=5,
                                    ngram_range=(1,4),
                                    max_features = 500
                                   )


# Split Data to train model
X_train, X_test, y_train, y_test = train_test_split(trainTitleFeatues[1], labels, test_size=0.25, random_state=RANDOM_SEED,stratify=labels) 


In [28]:
#Checking the shape 
testTitleFeatues[1].shape

(14832, 500)

In [29]:
lrModel= LogisticRegression(C=1.623776739188721, penalty='l1',
                            solver='liblinear', 
                            random_state=RANDOM_SEED,
                            max_iter=300,
                            )

In [30]:
lrModel.fit(X_train, y_train)

LogisticRegression(C=1.623776739188721, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=300, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=3971944, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [31]:
y_pred= lrModel.predict(X_test)

In [50]:
print("LogisticRegression Classification Report")
print(classification_report(y_pred,y_test))

LogisticRegression Classification Report
              precision    recall  f1-score   support

           F       0.55      0.79      0.65       233
          NF       0.88      0.71      0.79       517

    accuracy                           0.74       750
   macro avg       0.72      0.75      0.72       750
weighted avg       0.78      0.74      0.75       750



In [33]:
pd.DataFrame(lrModel.predict(testTitleFeatues[1]), index=test_df['id']).to_csv('S3971944-A2-predictions-11.csv')

In [34]:
rfcModel = RandomForestClassifier(bootstrap=True, 
                       n_estimators=300,
                       n_jobs=-1,
                       random_state=RANDOM_SEED)

In [35]:
rfcModel.fit(X_train, y_train)
y_pred= rfcModel.predict(X_test)

In [51]:
print("RandomForestClassifier Classification Report")
print(classification_report(y_pred,y_test))

RandomForestClassifier Classification Report
              precision    recall  f1-score   support

           F       0.55      0.79      0.65       233
          NF       0.88      0.71      0.79       517

    accuracy                           0.74       750
   macro avg       0.72      0.75      0.72       750
weighted avg       0.78      0.74      0.75       750



In [38]:
pd.DataFrame(rfcModel.predict(testTitleFeatues[1]), index=test_df['id']).to_csv('S3971944-A2-predictions-21.csv')

In [43]:
countDFuturistic = pd.DataFrame({"result":lrModel.predict(testTitleFeatues[1])})

In [52]:
print("Number of Tweets Classified as Futuristic and Non Futuristic")
countDFuturistic.value_counts()

Number of Tweets Classified as Futuristic and Non Futuristic


result
NF        10938
F          3894
dtype: int64

In [48]:
countDFuturistic[countDFuturistic['result']=='F'].value_counts()

result
F         3894
dtype: int64